## 혐오 댓글 생성하기

In [1]:
import os
import json
import math
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from argparse import Namespace
from tqdm.notebook import tqdm

### `Vocabulary`

In [2]:
class Vocabulary(object):
    """매핑을 위해 텍스트를 처리하고 어휘 사전을 만드는 클래스 """
    
    def __init__(self, token_to_idx=None):
        """
        매개변수:
            token_to_idx (dict): 기존 토큰-인덱스 매핑 딕셔너리
        """
        
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}

    def to_serializable(self):
        """ 직렬화할 수 있는 딕셔너리를 반환합니다 """
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        """ 직렬화된 딕셔너리에서 Vocabulary 객체를 만듭니다 """
        return cls(**contents)

    def add_token(self, token):
        """ 토큰을 기반으로 매핑 딕셔너리를 업데이트합니다

        매개변수:
            token (str): Vocabulary에 추가할 토큰
        반환값:
            index (int): 토큰에 상응하는 정수
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
            
    def add_many(self, tokens):
        """토큰 리스트를 Vocabulary에 추가합니다.
        
        매개변수:
            tokens (list): 문자열 토큰 리스트
        반환값:
            indices (list): 토큰 리스트에 상응되는 인덱스 리스트
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """토큰에 대응하는 인덱스를 추출합니다.
        
        매개변수:
            token (str): 찾을 토큰 
        반환값:
            index (int): 토큰에 해당하는 인덱스
        """
        return self._token_to_idx[token]

    def lookup_index(self, index):
        """ 인덱스에 해당하는 토큰을 반환합니다.
        
        매개변수: 
            index (int): 찾을 인덱스
        반환값:
            token (str): 인텍스에 해당하는 토큰
        에러:
            KeyError: 인덱스가 Vocabulary에 없을 때 발생합니다.
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

class SequenceVocabulary(Vocabulary):
    def __init__(self, tokenizer, token_to_idx=None):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        # KLUE RoBERTa 토크나이저에서 특수 토큰 설정
        self._mask_token = tokenizer.mask_token
        self._unk_token = tokenizer.unk_token
        self._begin_seq_token = tokenizer.cls_token
        self._end_seq_token = tokenizer.sep_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents
    
    @classmethod
    def from_serializable(cls, contents):
        # KLUE RoBERTa 토크나이저 초기화
        tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
        
        # token_to_idx만 추출하여 부모 클래스 초기화
        token_to_idx = contents.get('token_to_idx', {})
        return cls(tokenizer=tokenizer, token_to_idx=token_to_idx)

    def lookup_token(self, token):
        """ 토큰에 대응하는 인덱스를 추출합니다.
        토큰이 없으면 UNK 인덱스를 반환합니다.
        
        매개변수:
            token (str): 찾을 토큰 
        반환값:
            index (int): 토큰에 해당하는 인덱스
        노트:
            UNK 토큰을 사용하려면 (Vocabulary에 추가하기 위해)
            `unk_index`가 0보다 커야 합니다.
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

### `Vectorizer`

In [3]:
class CommentVectorizer(object):
    """ 어휘 사전을 생성하고 관리합니다 """
    
    def __init__(self, text_vocab, tokenizer):
        """
        매개변수:
            text_vocab (SequenceVocabulary): 댓글 텍스트의 토큰을 정수로 매핑합니다
        """
        self.tokenizer = tokenizer
        self.text_vocab = text_vocab

    def vectorize(self, text, vector_length=-1):
        """댓글 텍스트를 샘플과 타겟 벡터로 변환합니다
        
        매개변수:
            text (str): 벡터로 변경할 댓글 텍스트
            vector_length (int): 인덱스 벡터의 길이를 맞추기 위한 매개변수
        반환값:
            튜플: (from_vector, to_vector)
                from_vector (numpy.ndarray): 샘플 벡터
                to_vector (numpy.ndarray): 타겟 벡터
        """
        # RoBERTa tokenizer로 토큰화
        tokens = self.tokenizer.tokenize(text)
        
        # 1. 토큰을 인덱스로 변환
        # COMPLETE YOUR CODE - START
        token_indices = [self.text_vocab.lookup_token(token) for token in tokens]
        
        # COMPLETE YOUR CODE - END

        # 2. from_vector 생성
        # COMPLETE YOUR CODE - START
        from_indices = [self.text_vocab.begin_seq_index] + token_indices
        from_vector = np.zeros(vector_length, dtype=np.int64)
        from_vector[:len(from_indices)] = from_indices
        
        # COMPLETE YOUR CODE - END

        # 3. to_vector 생성
        # COMPLETE YOUR CODE - START
        to_indices = token_indices + [self.text_vocab.end_seq_index]
        to_vector = np.zeros(vector_length, dtype=np.int64)
        to_vector[:len(to_indices)] = to_indices
        
        # COMPLETE YOUR CODE - END

        return from_vector, to_vector

    @classmethod
    def from_dataframe(cls, df):
        """데이터셋 데이터프레임으로 객체를 초기화 합니다
        
        매개변수:
            df (pandas.DataFrame): 댓글 데이터프레임
        반환값:
            CommentVectorizer 객체
        """
        tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
        text_vocab = SequenceVocabulary(tokenizer=tokenizer)

        # 텍스트 어휘 사전 구축
        # COMPLETE YOUR CODE - START
        for text in df['text']:
            tokens = tokenizer.tokenize(text)
            text_vocab.add_many(tokens)
        
        # COMPLETE YOUR CODE - END

        return cls(text_vocab, tokenizer)

    @classmethod
    def from_serializable(cls, contents):
        """파일에서 CommentVectorizer 객체를 초기화합니다
        
        매개변수:
            contents (dict): CommentVectorizer를 위한 어휘 사전을 담은 딕셔너리
                이 딕셔너리는 `vectorizer.to_serializable()`를 사용해 만듭니다
        반환값:
            CommentVectorizer 객체
        """
        # KLUE RoBERTa 토크나이저 초기화
        tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
        
        text_vocab = SequenceVocabulary.from_serializable(contents['text_vocab'])
        return cls(text_vocab=text_vocab, tokenizer=tokenizer)

    def to_serializable(self):
        """직렬화된 결과를 반환합니다"""
        return {
            'text_vocab': self.text_vocab.to_serializable()
        }

## 파이토치 데이터셋 

In [4]:
class CommentDataset(Dataset):
    def __init__(self, comment_df, vectorizer):
        """
        매개변수:
            comment_df (pandas.DataFrame): 데이터셋
            vectorizer (CommentVectorizer): 데이터셋에서 만든 Vectorizer 객체
        """
        self.comment_df = comment_df
        self._vectorizer = vectorizer
        
        # 최대 시퀀스 길이 계산
        self._max_seq_length = max(len(self._vectorizer.tokenizer.tokenize(text))
            for text in self.comment_df['text']) + 2  # COMPLETE YOUR CODE

        # 데이터셋 분할
        self.train_df = self.comment_df[self.comment_df.split == 'train']
        self.train_size = len(self.train_df)

        self.val_df = self.comment_df[self.comment_df.split == 'val']
        self.validation_size = len(self.val_df)

        self.test_df = self.comment_df[self.comment_df.split == 'test']
        self.test_size = len(self.test_df)

        # 데이터셋 조회 딕셔너리 생성
        self._lookup_dict = {
            'train': (self.train_df, self.train_size),
            'val': (self.val_df, self.validation_size),
            'test': (self.test_df, self.test_size)
        }

        self.set_split('train')

    @classmethod
    def load_dataset_and_make_vectorizer(cls, dataset_csv):
        """데이터셋을 로드하고 새로운 Vectorizer를 만듭니다.
        
        매개변수:
            dataset_csv (str): 데이터셋의 위치
        반환값:
            CommentDataset 객체
        """
        comment_df = pd.read_csv(dataset_csv)
        vectorizer = CommentVectorizer.from_dataframe(comment_df)
        return cls(comment_df, vectorizer)
    
    @classmethod
    def load_dataset_and_load_vectorizer(cls, dataset_csv, vectorizer_filepath):
        """데이터셋과 캐싱된 Vectorizer 객체를 로드합니다.
        
        매개변수:
            dataset_csv (str): 데이터셋의 위치
            vectorizer_filepath (str): Vectorizer 객체의 저장 위치
        반환값:
            CommentDataset 객체
        """
        comment_df = pd.read_csv(dataset_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(comment_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """파일에서 Vectorizer 객체를 로드하는 정적 메서드
        
        매개변수:
            vectorizer_filepath (str): 직렬화된 Vectorizer 객체의 위치
        반환값:
            CommentVectorizer 객체
        """
        with open(vectorizer_filepath) as fp:
            return CommentVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """Vectorizer 객체를 json 형태로 디스크에 저장합니다.
        
        매개변수:
            vectorizer_filepath (str): Vectorizer 객체의 저장 위치
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """벡터 변환 객체를 반환합니다"""
        return self._vectorizer

    def set_split(self, split="train"):
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """파이토치 데이터셋의 주요 진입 메서드
        
        매개변수:
            index (int): 데이터 포인트에 대한 인덱스 
        반환값:
            데이터 포인트(x_data, y_target)를 담고 있는 딕셔너리
        """
        # COMPLETE YOUR CODE - START
        row = self._target_df.iloc[index]
        from_vector, to_vector = self._vectorizer.vectorize(row['text'], self._max_seq_length)

        # COMPLETE YOUR CODE - END

        return {
            'x_data': from_vector,
            'y_target': to_vector
        }
    
    def get_num_batches(self, batch_size):
        """배치 크기가 주어지면 데이터셋으로 만들 수 있는 배치 개수를 반환합니다.
        
        매개변수:
            batch_size (int)
        반환값:
            배치 개수
        """
        return len(self) // batch_size

### `DataLoader`

In [5]:
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"): 
    """
    파이토치 DataLoader를 감싸고 있는 제너레이터 함수.
    걱 텐서를 지정된 장치로 이동합니다.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

## 모델

In [6]:
class SingleLayerLSTMModel(nn.Module):
    def __init__(self, embedding_dim, num_embeddings, dropout_p,
                 hidden_size, embedding_path, padding_idx=0):
        """단층 LSTM 기반의 생성 모델
        
        매개변수:
            embedding_dim (int): KLUE RoBERTa 임베딩 차원
            num_embeddings (int): 임베딩 테이블 크기 (단어장 크기)
            hidden_size (int): LSTM의 은닉 상태 크기
            embedding_path (str): 정렬된 임베딩 파일 경로
            padding_idx (int): 패딩 토큰의 인덱스
            dropout_p (float): 드롭아웃 확률
        """
        super(SingleLayerLSTMModel, self).__init__()
        
        self.embedding = nn.Embedding.from_pretrained(
            torch.load(embedding_path),
            freeze=True,
            padding_idx=padding_idx
        )
        
        # COMPLETE YOUR CODE - START
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            batch_first=True
        )

        self.dropout = nn.Dropout(p=dropout_p)
        self.fc = nn.Linear(hidden_size, num_embeddings)
                
        # COMPLETE YOUR CODE - END
        
    def forward(self, x_in, apply_softmax=False):
        """순전파
        
        매개변수:
            x_in (torch.Tensor): 입력 텐서 (batch_size, sequence_length)
            apply_softmax (bool): 소프트맥스 적용 여부
        반환값:
            output (torch.Tensor): 출력 텐서 (batch_size, sequence_length, num_embeddings)
        """
        # COMPLETE YOUR CODE - START
        embedded = self.embedding(x_in)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = self.dropout(lstm_out)
        output = self.fc(lstm_out)

        if apply_softmax:
            output = F.softmax(output, dim=-1)
        
        # COMPLETE YOUR CODE - END
        return output


class SingleLayerGRUModel(nn.Module):
    def __init__(self, embedding_dim, num_embeddings, dropout_p,
                 hidden_size, embedding_path, padding_idx=0):
        """단층 GRU 기반의 생성 모델
        
        매개변수:
            embedding_dim (int): KLUE RoBERTa 임베딩 차원
            num_embeddings (int): 임베딩 테이블 크기 (단어장 크기)
            hidden_size (int): GRU의 은닉 상태 크기
            embedding_path (str): 정렬된 임베딩 파일 경로
            padding_idx (int): 패딩 토큰의 인덱스
            dropout_p (float): 드롭아웃 확률
        """
        super(SingleLayerGRUModel, self).__init__()
        
        self.embedding = nn.Embedding.from_pretrained(
            torch.load(embedding_path),
            freeze=True,
            padding_idx=padding_idx
        )
        
        # COMPLETE YOUR CODE - START
        self.gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            batch_first=True
        )

        self.dropout = nn.Dropout(p=dropout_p)
        self.fc = nn.Linear(hidden_size, num_embeddings)
                
        # COMPLETE YOUR CODE - END
        
    def forward(self, x_in, apply_softmax=False):
        """순전파
        
        매개변수:
            x_in (torch.Tensor): 입력 텐서 (batch_size, sequence_length)
            apply_softmax (bool): 소프트맥스 적용 여부
        반환값:
            output (torch.Tensor): 출력 텐서 (batch_size, sequence_length, num_embeddings)
        """
        # COMPLETE YOUR CODE - START
        embedded = self.embedding(x_in)
        gru_out, _ = self.gru(embedded)
        gru_out = self.dropout(gru_out)
        output = self.fc(gru_out)

        if apply_softmax:
            output = F.softmax(output, dim=-1)
        
        # COMPLETE YOUR CODE - END
        return output


class MultiLayerGRUModel(nn.Module):
    def __init__(self, embedding_dim, num_embeddings, dropout_p,
                 hidden_size, num_layers, embedding_path, padding_idx=0):
        """다층 GRU 기반의 생성 모델
        
        매개변수:
            embedding_dim (int): KLUE RoBERTa 임베딩 차원
            num_embeddings (int): 임베딩 테이블 크기 (단어장 크기)
            hidden_size (int): GRU의 은닉 상태 크기
            num_layers (int): GRU 층의 개수
            embedding_path (str): 정렬된 임베딩 파일 경로
            padding_idx (int): 패딩 토큰의 인덱스
            dropout_p (float): 드롭아웃 확률
        """
        super(MultiLayerGRUModel, self).__init__()
        
        self.embedding = nn.Embedding.from_pretrained(
            torch.load(embedding_path),
            freeze=True,
            padding_idx=padding_idx
        )
        
        # COMPLETE YOUR CODE - START
        self.gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout_p if num_layers > 1 else 0,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, num_embeddings)
                
        # COMPLETE YOUR CODE - END
        
    def forward(self, x_in, apply_softmax=False):
        """순전파
        
        매개변수:
            x_in (torch.Tensor): 입력 텐서 (batch_size, sequence_length)
            apply_softmax (bool): 소프트맥스 적용 여부
        반환값:
            output (torch.Tensor): 출력 텐서 (batch_size, sequence_length, num_embeddings)
        """
        # COMPLETE YOUR CODE - START
        embedded = self.embedding(x_in)
        gru_out, _ = self.gru(embedded)
        output = self.fc(gru_out)

        if apply_softmax:
            output = F.softmax(output, dim=-1)
            
        # COMPLETE YOUR CODE - END
        return output

### 헬퍼 함수

In [7]:
def normalize_sizes(y_pred, y_true):
    """텐서 크기 정규화

    매개변수:
        y_pred (torch.Tensor): 모델의 출력
            3차원 텐서이면 2차원 행렬로 변환합니다.
        y_true (torch.Tensor): 타깃 텐서
            2차원 텐서이면 1차원 벡터로 변환합니다.
    """
    if len(y_pred.size()) == 3:
        y_pred = y_pred.contiguous().view(-1, y_pred.size(2))
    if len(y_true.size()) == 2:
        y_true = y_true.contiguous().view(-1)
    return y_pred, y_true

def compute_accuracy(y_pred, y_true, mask_index):
    """정확도 계산

    매개변수:
        y_pred (torch.Tensor): 모델의 예측 결과
        y_true (torch.Tensor): 실제 정답
        mask_index (int): 마스크 토큰의 인덱스
    반환값:
        정확도 (float)
    """
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    _, y_pred_indices = y_pred.max(dim=1)

    correct_indices = torch.eq(y_pred_indices, y_true).float()
    valid_indices = torch.ne(y_true, mask_index).float()

    n_correct = (correct_indices * valid_indices).sum().item()
    n_valid = valid_indices.sum().item()

    return n_correct / n_valid * 100

def sequence_loss(y_pred, y_true, mask_index):
    """시퀀스 손실 계산

    매개변수:
        y_pred (torch.Tensor): 모델의 예측 결과
        y_true (torch.Tensor): 실제 정답
        mask_index (int): 마스크 토큰의 인덱스
    반환값:
        손실 값 (torch.Tensor)
    """
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    return F.cross_entropy(y_pred, y_true, ignore_index=mask_index)

def compute_perplexity(y_pred, y_true, mask_index):
    """Perplexity 계산 함수

    매개변수:
        y_pred (torch.Tensor): 모델의 예측 결과
        y_true (torch.Tensor): 실제 정답
        mask_index (int): 마스크 토큰의 인덱스
    반환값:
        perplexity (float): Perplexity 값
    """
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    loss = F.cross_entropy(y_pred, y_true, ignore_index=mask_index, reduction='sum')
    n_tokens = torch.ne(y_true, mask_index).sum().item()
    perplexity = math.exp(loss.item() / n_tokens)
    return perplexity

def make_train_state(args):
    """훈련 상태 초기화

    매개변수:
        args: 설정 값들이 담긴 Namespace 객체
    반환값:
        훈련 상태를 담은 딕셔너리
    """
    return {
        'stop_early': False,
        'early_stopping_step': 0,
        'early_stopping_best_val': 1e8,
        'learning_rate': args.learning_rate,
        'epoch_index': 0,
        'train_loss': [],
        'train_acc': [],
        'train_ppl': [],
        'val_loss': [],
        'val_acc': [],
        'val_ppl': [],
        'test_loss': -1,
        'test_acc': -1,
        'test_ppl': -1,
        'model_filename': args.model_state_file
    }

def update_train_state(args, model, train_state):
    """훈련 상태 업데이트

    매개변수:
        args: 설정 값들이 담긴 Namespace 객체
        model: 학습 중인 모델
        train_state: 현재 훈련 상태 딕셔너리
    반환값:
        업데이트된 훈련 상태 딕셔너리
    """
    # 첫 번째 에포크에서는 무조건 모델을 저장
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # 그 이후 에포크에서는 성능 개선 여부에 따라 모델 저장
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # 검증 손실이 증가하면 조기 종료 단계 증가
        if loss_t >= loss_tm1:
            train_state['early_stopping_step'] += 1
        # 검증 손실이 감소하면 모델 저장 및 조기 종료 단계 초기화
        else:
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t

            train_state['early_stopping_step'] = 0

        # 조기 종료 여부 판단
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

In [8]:
def save_and_align_roberta_embedding(vectorizer, args):
    """
    KLUE RoBERTa 임베딩을 저장하고 text_vocab에 맞게 정렬 후 재저장합니다.
    파일이 존재하면 기존 임베딩을 로드합니다.
    
    매개변수:
        vectorizer (CommentVectorizer): 데이터셋 기반의 Vectorizer 객체
        args (Namespace): 설정 값을 담은 객체 (embedding_path 포함)
    """
    # 이미 정렬된 임베딩 파일이 존재하면 로드
    if os.path.exists(args.embedding_path):
        print(f"임베딩 파일 로드 중: {args.embedding_path}")
        aligned_embedding = torch.load(args.embedding_path)
        return aligned_embedding

    print(f"새로운 KLUE RoBERTa 임베딩 생성 중...")
    
    # KLUE RoBERTa 토크나이저 및 모델 로드
    tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
    model_roberta = AutoModel.from_pretrained("klue/roberta-base")
    roberta_embedding = model_roberta.get_input_embeddings().weight.data

    # 새로운 정렬된 임베딩 초기화
    text_vocab = vectorizer.text_vocab
    aligned_embedding = torch.zeros(len(text_vocab), roberta_embedding.size(1))

    for idx in range(len(text_vocab)):
        token = text_vocab.lookup_index(idx)

        # 토큰 ID 가져와 임베딩 매핑
        if token in tokenizer.get_vocab():
            roberta_idx = tokenizer.convert_tokens_to_ids(token)
            aligned_embedding[idx] = roberta_embedding[roberta_idx]
        else:
            # KLUE RoBERTa에 없는 토큰은 랜덤 초기화
            aligned_embedding[idx] = torch.randn(roberta_embedding.size(1))

    # 정렬된 임베딩 저장
    torch.save(aligned_embedding, args.embedding_path)
    print(f"KLUE RoBERTa 임베딩 저장 완료 및 정렬됨: {args.embedding_path}")
    return aligned_embedding

## 설정

In [ ]:
print("1. 설정 및 초기화 중...")
args = Namespace(
    # 파일 경로
    comment_csv="hate_comments.csv",
    save_dir="model_storage/hate_comment_generation",
    vectorizer_file="vectorizer.json",
    embedding_path="model_storage/hate_comment_generation/roberta_embedding.pt",
    lstm_model_file="lstm_model.pth",
    single_gru_model_file="single_gru_model.pth",
    multi_gru_model_file="multi_gru_model.pth",
    
    # 모델 하이퍼파라미터
    embedding_dim=768,  # KLUE RoBERTa 임베딩 차원
    hidden_size=512,
    num_layers=2,
    dropout_p=0.1,
    
    # 훈련 하이퍼파라미터
    seed=1337,
    learning_rate=0.001,
    batch_size=64,
    num_epochs=100,
    early_stopping_criteria=5,
    
    # 실행 옵션
    catch_keyboard_interrupt=True,
    cuda=True,
    expand_filepaths_to_save_dir=True,
    reload_from_files=False,
)

model_files = {
    'SingleLayerLSTM': args.lstm_model_file,
    'SingleLayerGRU': args.single_gru_model_file,
    'MultiLayerGRU': args.multi_gru_model_file
}

# 경로 확장
if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)
    for model_name in model_files:
        model_files[model_name] = os.path.join(args.save_dir, model_files[model_name])
        
    print("\n파일 경로:")
    print(f"Vectorizer: {args.vectorizer_file}")
    print("\n저장될 모델 파일:")
    for model_name, path in model_files.items():
        print(f"{model_name}: {path}")

# CUDA 설정
if not torch.cuda.is_available():
    args.cuda = False
    
args.device = torch.device("cuda" if args.cuda else "cpu")
print("CUDA 사용 여부: {}".format(args.cuda))

# 재현성을 위한 시드 설정 및 디렉토리 생성
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

set_seed_everywhere(args.seed, args.cuda)
handle_dirs(args.save_dir)

## 훈련 반복

In [ ]:
print("\n2. 데이터 로드 중...")
print(f"데이터셋 파일: {args.comment_csv}")

# 데이터셋과 벡터라이저 로드 또는 생성
if args.reload_from_files and os.path.exists(args.vectorizer_file):
    # 체크포인트에서 데이터 로드
    print("\n체크포인트에서 데이터 로드 중...")
    dataset = CommentDataset.load_dataset_and_load_vectorizer(
        args.comment_csv, args.vectorizer_file)
else:
    # 새로운 데이터셋과 벡터라이저 생성
    print("\n데이터셋 생성 중...")
    dataset = CommentDataset.load_dataset_and_make_vectorizer(args.comment_csv)
    dataset.save_vectorizer(args.vectorizer_file)

vectorizer = dataset.get_vectorizer()
print(f"데이터셋 크기: {len(dataset):,} 샘플")
print(f"어휘 사전 크기: {len(vectorizer.text_vocab):,} 토큰")

print("\n3. CUDA 설정 확인 중...")
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")
print(f"학습 장치: {args.device}")

# KLUE RoBERTa 임베딩 저장 및 정렬
save_and_align_roberta_embedding(vectorizer, args)

print("\n4. 모델 초기화 중...")
# 모델 초기화 및 학습
models = {
    'SingleLayerLSTM': SingleLayerLSTMModel(
        embedding_dim=args.embedding_dim,
        num_embeddings=len(vectorizer.text_vocab),
        hidden_size=args.hidden_size,
        padding_idx=vectorizer.text_vocab.mask_index,
        dropout_p=args.dropout_p,
        embedding_path=args.embedding_path
    ),
    'SingleLayerGRU': SingleLayerGRUModel(
        embedding_dim=args.embedding_dim,
        num_embeddings=len(vectorizer.text_vocab),
        hidden_size=args.hidden_size,
        padding_idx=vectorizer.text_vocab.mask_index,
        dropout_p=args.dropout_p,
        embedding_path=args.embedding_path
    ),
    'MultiLayerGRU': MultiLayerGRUModel(
        embedding_dim=args.embedding_dim,
        num_embeddings=len(vectorizer.text_vocab),
        hidden_size=args.hidden_size,
        num_layers=args.num_layers,
        padding_idx=vectorizer.text_vocab.mask_index,
        dropout_p=args.dropout_p,
        embedding_path=args.embedding_path
    )
}

print("\n5. 모델 학습 시작...")
train_states = {}

for model_name, model in models.items():
    print(f"\n=== {model_name} 모델 학습 ===")
    model = model.to(args.device)
    
    # 모델별 train_state 초기화
    args_copy = copy.deepcopy(args)
    args_copy.model_state_file = model_files[model_name]  # 각 모델별 저장 경로 설정
    train_state = make_train_state(args_copy)
    train_states[model_name] = train_state
    
    print(f"모델이 저장될 경로: {train_state['model_filename']}")
    
    # 옵티마이저 설정
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer=optimizer, mode='min', factor=0.5, patience=1
    )

    # tqdm을 사용한 진행 상태 막대 설정
    epoch_bar = tqdm(desc='에포크 진행', total=args.num_epochs, position=0)
    dataset.set_split('train')
    train_bar = tqdm(desc='훈련 중', total=dataset.get_num_batches(args.batch_size),
                     position=1, leave=True)
    dataset.set_split('val')
    val_bar = tqdm(desc='검증 중', total=dataset.get_num_batches(args.batch_size),
                   position=1, leave=True)

    try:
        for epoch_index in range(args.num_epochs):
            train_state['epoch_index'] = epoch_index

            # 훈련 세트에 대한 순회
            model.train()
            dataset.set_split('train')
            running_loss = 0.0
            running_acc = 0.0
            running_ppl = 0.0
            batch_generator = generate_batches(dataset, args.batch_size, device=args.device)

            # 진행 상태 막대 초기화
            train_bar.reset()

            for batch_index, batch_dict in enumerate(batch_generator):
                # 단계 1. 그레이디언트를 0으로 초기화합니다
                optimizer.zero_grad()

                # 단계 2. 출력을 계산합니다
                y_pred = model(
                    x_in=batch_dict['x_data']
                )

                # 단계 3. 손실을 계산합니다
                loss = sequence_loss(y_pred, batch_dict['y_target'],
                                     vectorizer.text_vocab.mask_index)

                # 단계 4. 손실을 사용해 그레이디언트를 계산합니다
                loss.backward()

                # 단계 5. 옵티마이저로 가중치를 업데이트합니다
                optimizer.step()

                # 이동 손실과 이동 정확도를 계산합니다
                running_loss += (loss.item() - running_loss) / (batch_index + 1)
                acc_t = compute_accuracy(y_pred, batch_dict['y_target'],
                                         vectorizer.text_vocab.mask_index)
                running_acc += (acc_t - running_acc) / (batch_index + 1)
                ppl_t = compute_perplexity(y_pred, batch_dict['y_target'],
                                           vectorizer.text_vocab.mask_index)
                running_ppl += (ppl_t - running_ppl) / (batch_index + 1)

                # 진행 상태 막대 업데이트
                train_bar.set_postfix(loss=running_loss, acc=running_acc, ppl=running_ppl)
                train_bar.update()

            train_state['train_loss'].append(running_loss)
            train_state['train_acc'].append(running_acc)
            train_state['train_ppl'].append(running_ppl)

            # 검증 세트에 대한 순회
            model.eval()
            dataset.set_split('val')
            running_loss = 0.0
            running_acc = 0.0
            running_ppl = 0.0
            batch_generator = generate_batches(dataset, args.batch_size, device=args.device)

            # 진행 상태 막대 초기화
            val_bar.reset()

            with torch.no_grad():
                for batch_index, batch_dict in enumerate(batch_generator):
                    # 단계 1. 출력을 계산합니다
                    y_pred = model(
                        x_in=batch_dict['x_data']
                    )

                    # 단계 2. 손실을 계산합니다
                    loss = sequence_loss(y_pred, batch_dict['y_target'],
                                         vectorizer.text_vocab.mask_index)

                    # 단계 3. 이동 손실과 이동 정확도를 계산합니다
                    running_loss += (loss.item() - running_loss) / (batch_index + 1)
                    acc_t = compute_accuracy(y_pred, batch_dict['y_target'],
                                             vectorizer.text_vocab.mask_index)
                    running_acc += (acc_t - running_acc) / (batch_index + 1)
                    ppl_t = compute_perplexity(y_pred, batch_dict['y_target'],
                                               vectorizer.text_vocab.mask_index)
                    running_ppl += (ppl_t - running_ppl) / (batch_index + 1)

                    # 진행 상태 막대 업데이트
                    val_bar.set_postfix(loss=running_loss, acc=running_acc, ppl=running_ppl)
                    val_bar.update()

            train_state['val_loss'].append(running_loss)
            train_state['val_acc'].append(running_acc)
            train_state['val_ppl'].append(running_ppl)

            # 학습 상태 업데이트 및 조기 종료 확인
            train_state = update_train_state(args=args_copy, model=model, 
                                           train_state=train_state)
            scheduler.step(train_state['val_loss'][-1])

            # 에포크 진행 상태 막대 업데이트
            epoch_bar.update()
            epoch_bar.set_postfix(epoch=epoch_index)

            # 조기 종료 여부 확인
            if train_state['stop_early']:
                print(f"\n{model_name} 모델 조기 종료!")
                break

        train_states[model_name] = train_state
        print(f"\n{model_name} 모델 학습 완료!")

    except KeyboardInterrupt:
        print(f"\n{model_name} 모델 학습 중단!")

## 테스트 세트 평가

In [ ]:
print("\n6. 각 모델 테스트 시작...")
test_results = {}

for model_name, model in models.items():
    print(f"\n=== {model_name} 모델 테스트 ===")
    
    # 저장된 모델 가중치를 로드하고 장치로 이동
    model.load_state_dict(torch.load(train_states[model_name]['model_filename']))
    model = model.to(args.device)
    model.eval()

    # 테스트 데이터셋 설정
    dataset.set_split('test')
    batch_generator = generate_batches(dataset, args.batch_size, device=args.device)

    # 진행 상태 막대 설정
    test_bar = tqdm(desc=f'{model_name} 테스트 중', total=dataset.get_num_batches(args.batch_size), position=0)

    test_loss = 0.0
    test_acc = 0.0
    test_ppl = 0.0

    with torch.no_grad():
        for batch_index, batch_dict in enumerate(batch_generator):
            # 모델 예측
            y_pred = model(
                x_in=batch_dict['x_data']
            )

            # 손실 계산
            loss = sequence_loss(y_pred, batch_dict['y_target'],
                                 vectorizer.text_vocab.mask_index)

            # 정확도 계산
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'],
                                     vectorizer.text_vocab.mask_index)

            # Perplexity 계산
            ppl_t = compute_perplexity(y_pred, batch_dict['y_target'],
                                       vectorizer.text_vocab.mask_index)

            # 손실과 정확도 누적
            test_loss += (loss.item() - test_loss) / (batch_index + 1)
            test_acc += (acc_t - test_acc) / (batch_index + 1)
            test_ppl += (ppl_t - test_ppl) / (batch_index + 1)

            # 진행 상태 막대 업데이트
            test_bar.set_postfix(loss=test_loss, acc=test_acc, ppl=test_ppl)
            test_bar.update()

    test_results[model_name] = {
        'loss': test_loss,
        'accuracy': test_acc,
        'perplexity': test_ppl
    }
    
    train_states[model_name]['test_loss'] = test_loss
    train_states[model_name]['test_acc'] = test_acc
    train_states[model_name]['test_ppl'] = test_ppl

    test_bar.close()

    print(f"\n{model_name} 테스트 결과:")
    print(f"Loss: {test_loss:.4f}")
    print(f"Accuracy: {test_acc:.2f}%")
    print(f"Perplexity: {test_ppl:.2f}")

# 모델 성능 비교 출력
print("\n=== 모델 성능 비교 ===")
print("="*60)
print(f"{'모델':^20} {'Loss':^12} {'Accuracy':^12} {'Perplexity':^12}")
print("="*60)
for model_name, results in test_results.items():
    print(f"{model_name:^20} {results['loss']:^12.4f} {results['accuracy']:^12.2f} {results['perplexity']:^12.2f}")
print("="*60)